# A Guided Tour of Ray Core: Remote Stateful Classes

© 2019-2022, Anyscale. All Rights Reserved

### Learning objectives
In this this tutorial, we'll discuss Ray Actors and learn about:
 * How Ray Actors work
 * How to write a stateful Ray Actor
 * How Ray Actors can be writen as a statful distributed service

[*Remote Classes*](https://docs.ray.io/en/latest/walkthrough.html#remote-classes-actors)
involve using a `@ray.remote` decorator on a class. 

This implements an [*actor*](https://patterns.eecs.berkeley.edu/?page_id=258) pattern, with properties: *stateful*, *message-passing semantics*

Actors are extremely powerful. They allow you to take a Python class and instantiate it as a stateful microservice that can be queried from other actors and tasks and even other Python applications.

When you instantiate a remote Actor, a separate worker process is created as a worker process and becomes an Actor process on the worker node, for the purpose of running methods called on the actor. Other Ray tasks and actors can invoke its methods on that process, mutating its internal state. Actors can also be terminated manually if needed. The examples code below show all these cases.

<img src="images/ray_worker_actor_1.png" height="40%" width="70%">
<img src="images/ray_worker_actor_2.png" height="40%" width="70%">

---

First, let's start Ray…

In [1]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [2]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-17_14-13-36_212995_43088/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-17_14-13-36_212995_43088/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-05-17_14-13-36_212995_43088', 'metrics_export_port': 62091, 'gcs_address': '127.0.0.1:61279', 'address': '127.0.0.1:61279', 'node_id': '47d9a2d97ac61d1f3ec9b8be624073b689ca0ec1443c183e8f9d20c9'})


In [4]:
print(f"Dashboard url: http://{context.address_info['webui_url']}")

Dashboard url: http://127.0.0.1:8265


## 3. Remote Class as a Stateful Actor Pattern

To start, we'll define a class and use the decorator: `@ray.remote`

Let's use Python class and convert that to a remote Actor class actor service as a Parameter Server. This is a common example in machine learning where you have a central Parameter server updating gradients from other worker processes computing individual gradients. 

<img src="https://terrytangyuan.github.io/img/inblog/mpi-operator-1.png" width="60%" height="30%">

In [5]:
@ray.remote
class ParameterSever:
    def __init__(self):
        # Initialized our gradients to zero
        self.params = np.zeros(10)

    def get_params(self):
        # Return current gradients
        return self.params

    def update_params(self, grad):
        # Update the gradients 
        self.params -= grad

Define worker or task as a function for a remote Worker process. This could be a machine learning objective function that computes gradients and sends them to the parameter server.

In [6]:
@ray.remote
def worker(ps):
    # Iterate over some epoch
    for i in range(25):
        time.sleep(1.5)  # this could be your loss function computing gradients
        grad = np.ones(10)
        # update the gradients in the parameter server
        ps.update_params.remote(grad)

Start our Parameter Server actor. This will be scheduled as a process on a remote Ray Worker. You invoke its `ActorClass.remote(...)` to instantiate an Actor instance of that type.

In [7]:
param_server = ParameterSever.remote()
param_server

Actor(ParameterSever, f0f29fd04d0e89d3b7cc696e01000000)

Let's get the initial values of the parameter server

In [8]:
print(f"Initial params: {ray.get(param_server.get_params.remote())}")

Initial params: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Create Workers Nodes Computing Gradients
Let's create three separate workers as our machine learning tasks that compute gradients.
These will be scheduled as tasks on a Ray cluster.

You can use list comprehension. Quite Pythonic!

If we need more workers to scale, we can always bump them up.

**Note**: That we are sending the `parameter_server` as an argument to the remote
worker task. Ray will resolve this.

In [9]:
[worker.remote(param_server) for _ in range(3)]

[ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000),
 ObjectRef(32d950ec0ccf9d2affffffffffffffffffffffff0100000001000000),
 ObjectRef(e0dc174c83599034ffffffffffffffffffffffff0100000001000000)]

Now, let's iterate over a loop and query the Parameter Server 
as the workers are running independently and updating the gradients

In [10]:
for _i in range(20):
    print(f"Updated params: {ray.get(param_server.get_params.remote())}")
    time.sleep(1)

Updated params: [-9. -9. -9. -9. -9. -9. -9. -9. -9. -9.]
Updated params: [-12. -12. -12. -12. -12. -12. -12. -12. -12. -12.]
Updated params: [-12. -12. -12. -12. -12. -12. -12. -12. -12. -12.]
Updated params: [-15. -15. -15. -15. -15. -15. -15. -15. -15. -15.]
Updated params: [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18.]
Updated params: [-18. -18. -18. -18. -18. -18. -18. -18. -18. -18.]
Updated params: [-21. -21. -21. -21. -21. -21. -21. -21. -21. -21.]
Updated params: [-24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
Updated params: [-24. -24. -24. -24. -24. -24. -24. -24. -24. -24.]
Updated params: [-27. -27. -27. -27. -27. -27. -27. -27. -27. -27.]
Updated params: [-30. -30. -30. -30. -30. -30. -30. -30. -30. -30.]
Updated params: [-30. -30. -30. -30. -30. -30. -30. -30. -30. -30.]
Updated params: [-33. -33. -33. -33. -33. -33. -33. -33. -33. -33.]
Updated params: [-36. -36. -36. -36. -36. -36. -36. -36. -36. -36.]
Updated params: [-36. -36. -36. -36. -36. -36. -36. -36. -

### Look at the Ray Dashboard

You should see Actors running as process on the workers nodes
 * Parameter Server
 
Also, click on the `Logical View` to view more metrics and data on individual Ray Actors

Finally, shutdown Ray

In [11]:
ray.shutdown()

### Exercises

1. Add a remote class, that keeps states by keeping track in a Actor class instance (may be only in memory)
2. Implement methods that alters the state
3. Instantiate it and call its methods

---
## References

 * [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)
 * [Advanced Patterns and Anti-Patterns in Ray](https://docs.ray.io/en/latest/ray-design-patterns/index.htmlhttps://docs.ray.io/en/latest/ray-design-patterns/index.html)